# Face Detection на WIDER FACE

Быстрая проверка качества детекции лиц. Используем Haar Cascade как baseline, потом можно будет сравнить с более современными методами.

**Что делаем:**
- Берем WIDER FACE датасет
- Детектим лица Haar Cascade'ом
- Считаем метрики (precision/recall)
- Смотрим на скорость работы
- Визуализируем результаты


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
import glob
import os
from pathlib import Path
import json

# Убираем лишние предупреждения
import warnings
warnings.filterwarnings('ignore')

# Создаем папку для результатов
os.makedirs('results', exist_ok=True)

# Настройки для красивого отображения
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['image.cmap'] = 'gray'


## Загрузка данных

WIDER FACE лежит в `data/images/`. Загружаем все jpg файлы.


In [ ]:
def load_images(data_dir="data/images"):
    """Загружаем все jpg файлы из папки"""
    images = glob.glob(f"{data_dir}/*.jpg")
    print(f"Найдено {len(images)} изображений")
    return images

def load_image(path):
    """Загружаем одно изображение"""
    return cv2.imread(path)

def load_annotations(annos_file="data/annos.json"):
    """Загружаем аннотации из JSON файла"""
    with open(annos_file, 'r') as f:
        data = json.load(f)
    
    annotations = {}
    for item in data:
        img_path = item['img_path'].replace('\\', '/')  # Исправляем пути для Unix
        bboxes = item['annotations']['bbox']
        # Фильтруем только валидные аннотации (invalid=0)
        valid_boxes = []
        for i, bbox in enumerate(bboxes):
            if item['annotations']['invalid'][i] == 0:
                valid_boxes.append(bbox)
        annotations[img_path] = valid_boxes
    
    print(f"Загружено аннотаций для {len(annotations)} изображений")
    return annotations

# Загружаем данные
image_paths = load_images()
annotations = load_annotations()


## Детектор лиц

Используем стандартный Haar Cascade из OpenCV. Быстро, но не самый точный.


In [ ]:
# Загружаем Haar Cascade классификатор
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_faces(image, scale_factor=1.1, min_neighbors=5, min_size=(30, 30)):
    """Детектим лица на изображении"""
    if image is None:
        return []
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=scale_factor, 
                                        minNeighbors=min_neighbors, minSize=min_size)
    return faces


## Метрики

Считаем IoU, precision, recall. Без этого никак.


In [ ]:
def iou(box1, box2):
    """Считаем IoU между двумя прямоугольниками"""
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    
    # Пересечение
    x_left = max(x1, x2)
    y_top = max(y1, y2)
    x_right = min(x1 + w1, x2 + w2)
    y_bottom = min(y1 + h1, y2 + h2)
    
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    
    intersection = (x_right - x_left) * (y_bottom - y_top)
    union = w1 * h1 + w2 * h2 - intersection
    
    return intersection / union if union > 0 else 0.0

def match_boxes(pred_boxes, gt_boxes, iou_threshold=0.5):
    """Сопоставляем предсказания с ground truth"""
    matches = []
    used_gt = set()
    
    for pred_idx, pred_box in enumerate(pred_boxes):
        best_iou = 0
        best_gt_idx = -1
        
        for gt_idx, gt_box in enumerate(gt_boxes):
            if gt_idx in used_gt:
                continue
                
            current_iou = iou(pred_box, gt_box)
            if current_iou > best_iou and current_iou >= iou_threshold:
                best_iou = current_iou
                best_gt_idx = gt_idx
        
        if best_gt_idx != -1:
            matches.append((pred_idx, best_gt_idx, best_iou))
            used_gt.add(best_gt_idx)
    
    matched_pred = {match[0] for match in matches}
    unmatched_pred = [i for i in range(len(pred_boxes)) if i not in matched_pred]
    unmatched_gt = [i for i in range(len(gt_boxes)) if i not in used_gt]
    
    return matches, unmatched_pred, unmatched_gt

def calculate_metrics(pred_boxes, gt_boxes, iou_threshold=0.5):
    """Считаем precision, recall, f1"""
    if len(gt_boxes) == 0:
        return {"precision": 1.0 if len(pred_boxes) == 0 else 0.0, 
                "recall": 1.0, "f1": 1.0, "tp": 0, "fp": len(pred_boxes), "fn": 0}
    
    if len(pred_boxes) == 0:
        return {"precision": 0.0, "recall": 0.0, "f1": 0.0, "tp": 0, "fp": 0, "fn": len(gt_boxes)}
    
    matches, unmatched_pred, unmatched_gt = match_boxes(pred_boxes, gt_boxes, iou_threshold)
    
    tp = len(matches)
    fp = len(unmatched_pred)
    fn = len(unmatched_gt)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    return {"precision": precision, "recall": recall, "f1": f1, "tp": tp, "fp": fp, "fn": fn}


## Визуализация

Рисуем bounding boxes и смотрим что получилось.


In [ ]:
def draw_boxes(image, boxes, color=(0, 255, 0), thickness=2):
    """Рисуем bounding boxes на изображении"""
    result = image.copy()
    for x, y, w, h in boxes:
        cv2.rectangle(result, (x, y), (x + w, y + h), color, thickness)
    return result

def show_detection(image, pred_boxes, gt_boxes=None, save_path=None):
    """Показываем результаты детекции"""
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    if gt_boxes is not None:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        # Предсказания
        pred_img = draw_boxes(image_rgb, pred_boxes, color=(0, 255, 0))
        ax1.imshow(pred_img)
        ax1.set_title(f"Предсказания ({len(pred_boxes)})")
        ax1.axis('off')
        
        # Ground truth
        gt_img = draw_boxes(image_rgb, gt_boxes, color=(255, 0, 0))
        ax2.imshow(gt_img)
        ax2.set_title(f"Ground Truth ({len(gt_boxes)})")
        ax2.axis('off')
    else:
        plt.figure(figsize=(10, 8))
        pred_img = draw_boxes(image_rgb, pred_boxes, color=(0, 255, 0))
        plt.imshow(pred_img)
        plt.title(f"Детекция лиц ({len(pred_boxes)})")
        plt.axis('off')
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"Сохранено: {save_path}")
    
    plt.show()


## Тестирование

Проверяем на нескольких изображениях как работает детектор.


In [ ]:
# Тестируем на первых 5 изображениях
test_images = image_paths[:5]

for i, img_path in enumerate(test_images):
    print(f"\n--- Изображение {i+1}: {os.path.basename(img_path)} ---")
    
    # Загружаем изображение
    image = load_image(img_path)
    if image is None:
        print("Не удалось загрузить изображение")
        continue
    
    # Получаем ground truth
    img_key = f"images/{os.path.basename(img_path)}"
    gt_faces = annotations.get(img_key, [])
    
    # Детектируем лица
    start_time = time.time()
    faces = detect_faces(image)
    detection_time = time.time() - start_time
    
    print(f"Найдено лиц: {len(faces)}")
    print(f"Ground truth: {len(gt_faces)}")
    print(f"Время детекции: {detection_time:.3f} сек")
    print(f"FPS: {1/detection_time:.1f}")
    
    # Считаем метрики
    if gt_faces:
        metrics = calculate_metrics(faces, gt_faces)
        print(f"Precision: {metrics['precision']:.3f}")
        print(f"Recall: {metrics['recall']:.3f}")
        print(f"F1: {metrics['f1']:.3f}")
    
    # Показываем результат
    show_detection(image, faces, gt_faces, save_path=f"results/detection_{i+1}.png")


## Полная оценка

Теперь прогоняем весь датасет и считаем общие метрики.


In [ ]:
def evaluate_dataset(image_paths, annotations, max_images=None):
    """Оцениваем качество детекции на всем датасете"""
    if max_images:
        image_paths = image_paths[:max_images]
    
    total_faces = 0
    total_gt_faces = 0
    total_time = 0
    all_metrics = []
    
    print(f"Обрабатываем {len(image_paths)} изображений...")
    
    for i, img_path in enumerate(image_paths):
        if i % 50 == 0:
            print(f"Обработано: {i}/{len(image_paths)}")
        
        # Загружаем изображение
        image = load_image(img_path)
        if image is None:
            continue
        
        # Получаем ground truth
        img_key = f"images/{os.path.basename(img_path)}"
        gt_faces = annotations.get(img_key, [])
        
        # Детектируем лица
        start_time = time.time()
        faces = detect_faces(image)
        detection_time = time.time() - start_time
        
        total_faces += len(faces)
        total_gt_faces += len(gt_faces)
        total_time += detection_time
        
        # Считаем метрики
        metrics = calculate_metrics(faces, gt_faces)
        all_metrics.append(metrics)
    
    # Общие результаты
    avg_faces_per_image = total_faces / len(image_paths)
    avg_gt_faces_per_image = total_gt_faces / len(image_paths)
    avg_time_per_image = total_time / len(image_paths)
    fps = 1 / avg_time_per_image
    
    # Средние метрики
    avg_precision = np.mean([m['precision'] for m in all_metrics])
    avg_recall = np.mean([m['recall'] for m in all_metrics])
    avg_f1 = np.mean([m['f1'] for m in all_metrics])
    
    print(f"\n=== РЕЗУЛЬТАТЫ ===")
    print(f"Обработано изображений: {len(image_paths)}")
    print(f"Всего найдено лиц: {total_faces}")
    print(f"Всего GT лиц: {total_gt_faces}")
    print(f"Среднее лиц на изображение: {avg_faces_per_image:.2f}")
    print(f"Среднее GT лиц на изображение: {avg_gt_faces_per_image:.2f}")
    print(f"Среднее время на изображение: {avg_time_per_image:.3f} сек")
    print(f"Средний FPS: {fps:.1f}")
    print(f"Средняя Precision: {avg_precision:.3f}")
    print(f"Средняя Recall: {avg_recall:.3f}")
    print(f"Средняя F1: {avg_f1:.3f}")
    
    return {
        "total_images": len(image_paths),
        "total_faces": total_faces,
        "total_gt_faces": total_gt_faces,
        "avg_faces_per_image": avg_faces_per_image,
        "avg_gt_faces_per_image": avg_gt_faces_per_image,
        "avg_time_per_image": avg_time_per_image,
        "fps": fps,
        "avg_precision": avg_precision,
        "avg_recall": avg_recall,
        "avg_f1": avg_f1
    }

# Запускаем оценку (ограничиваем 100 изображениями для скорости)
results = evaluate_dataset(image_paths, annotations, max_images=100)


## Итоги

Haar Cascade работает быстро, но точность не самая высокая. Для продакшена лучше использовать более современные методы.


In [ ]:
# Сохраняем результаты
with open("results/evaluation_results.json", "w") as f:
    json.dump(results, f, indent=2)

print("Результаты сохранены в results/evaluation_results.json")

# Создаем краткий отчет
report = f"""
# Отчет по детекции лиц

## Метод: Haar Cascade (OpenCV)

## Результаты на {results['total_images']} изображениях:

- **Скорость**: {results['fps']:.1f} FPS
- **Precision**: {results['avg_precision']:.3f}
- **Recall**: {results['avg_recall']:.3f}
- **F1-score**: {results['avg_f1']:.3f}

## Статистика:
- Найдено лиц: {results['total_faces']}
- Ground truth лиц: {results['total_gt_faces']}
- Среднее лиц на изображение: {results['avg_faces_per_image']:.2f}

## Выводы:
Haar Cascade показывает базовые результаты. Для улучшения качества рекомендуется использовать более современные методы (RetinaFace, MTCNN, YOLO).
"""

with open("results/report.md", "w", encoding="utf-8") as f:
    f.write(report)

print("Отчет сохранен в results/report.md")
